<a href="https://colab.research.google.com/github/CarlosZuluagaU/KaggleUdeACarlosZuluaga/blob/main/99_modelo_soluci%C3%B3nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo XGBoost mejorado

es un algoritmo de gradient boosting basado en árboles de decisión, optimizado para resolver problemas complejos de clasificación. En tu proyecto, este modelo analiza múltiples variables de los estudiantes (características académicas, socioeconómicas, personales, etc.) para predecir el nivel de RENDIMIENTO_GLOBAL, una etiqueta categórica definida por la competencia.

XGBoost es uno de los modelos más utilizados en competencias de Kaggle por su:

Excelente rendimiento en tareas de clasificación multiclase

Capacidad para manejar datos con ruido y variables heterogéneas

Regularización que reduce sobreajuste

Velocidad mucho mayor que Random Forest y otros métodos tradicionales

Gran capacidad de ajuste mediante hiperparámetros

El objetivo principal del modelo es:

Predecir con la mayor exactitud posible el RENDIMIENTO_GLOBAL de cada estudiante en el archivo de test.

In [1]:
!pip install rlxutils


  Preparing metadata (setup.py) ... done
  Created wheel for rlxutils: filename=rlxutils-0.1.10-py3-none-any.whl size=11100 sha256=3754b04087ef42a896367512ce6c4c3cc73c9907250ffe6a6cb1a30eebfe360a
  Stored in directory: /root/.cache/pip/wheels/fa/8f/e3/7c998c7a46a028383fd3c53494f18d81d52b7368c6b33e4ab6
Successfully built rlxutils


In [12]:
# Instalar la librería de Kaggle
!pip install kaggle

import os

# Configurar ruta donde está tu kaggle.json (en el directorio actual)
os.environ['KAGGLE_CONFIG_DIR'] = '.'

# Ajustar permisos del archivo kaggle.json
!chmod 600 ./kaggle.json

# Descargar dataset de la competencia
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

# Crear carpeta "data" y descomprimir ahí
!mkdir -p data
!unzip udea*.zip -d data


  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.40GB/s]
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: data/submission_example.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


In [13]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [14]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia


udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!pip install -q kaggle
import os

# Configura la ruta del token
os.environ['KAGGLE_CONFIG_DIR'] = '.'

# Permisos de seguridad para el archivo
!chmod 600 ./kaggle.json

# Descarga el dataset del concurso
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia


401 Client Error: Unauthorized for url: https://www.kaggle.com/api/v1/competitions/data/download-all/udea-ai-4-eng-20252-pruebas-saber-pro-colombia


In [5]:
!pip install xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)


In [15]:
print("Cargando datasets...")
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Guardamos los IDs del test para el archivo final, ya que los eliminaremos al procesar
test_ids = test['ID']

print(f"Dimensiones Train: {train.shape}")
print(f"Dimensiones Test: {test.shape}")

Cargando datasets...
Dimensiones Train: (692500, 21)
Dimensiones Test: (296786, 20)


In [16]:
# 1. Mapeo del Target (Variable Objetivo)
target_map = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}
inv_target_map = {v: k for k, v in target_map.items()}

# 2. Mapeos Ordinales (Educación y Estrato)
# Asignamos valores numéricos crecientes según el nivel
edu_map = {
    'No sabe': 0, 'No Aplica': 0, 'Ninguno': 0,
    'Primaria incompleta': 1, 'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
    'Postgrado': 9
}

estrato_map = {
    'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3,
    'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6,
    'Sin Estrato': 0
}

print("Diccionarios de mapeo definidos correctamente.")

Diccionarios de mapeo definidos correctamente.


In [17]:
# --- BLOQUE 4 MEJORADO (FEATURE ENGINEERING) ---

def preprocess_data(df, is_train=True):
    # Trabajamos sobre una copia
    df_proc = df.copy()

    # --- 1. Mapeos Ordinales (Igual que antes) ---
    cols_edu = ['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']
    for col in cols_edu:
        if col in df_proc.columns:
            df_proc[col] = df_proc[col].map(edu_map).fillna(1) # Rellenar con 1 (Primaria) en vez de -1 ayuda más

    if 'F_ESTRATOVIVIENDA' in df_proc.columns:
        df_proc['F_ESTRATOVIVIENDA'] = df_proc['F_ESTRATOVIVIENDA'].map(estrato_map).fillna(2) # Rellenar con estrato 2 (moda)

    # --- 2. Variables Binarias ---
    binarias = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENECOMPUTADOR', 'E_PAGOMATRICULAPROPIO']
    for col in binarias:
        if col in df_proc.columns:
            df_proc[col] = df_proc[col].map({'Si': 1, 'No': 0, 'S': 1, 'N': 0}).fillna(0)

    # --- 3. NUEVAS VARIABLES (Aquí está el truco para subir el accuracy) ---

    # A. Índice de Riqueza Tecnológica (Suma de posesiones)
    # Cuantas más cosas tenga, más probable es que tenga mejor rendimiento
    cols_bienes = [c for c in ['F_TIENEINTERNET', 'F_TIENECOMPUTADOR', 'F_TIENELAVADORA'] if c in df_proc.columns]
    df_proc['SCORE_BIENES'] = df_proc[cols_bienes].sum(axis=1)

    # B. Capital Cultural en Casa (Suma educación padres)
    if 'F_EDUCACIONPADRE' in df_proc.columns and 'F_EDUCACIONMADRE' in df_proc.columns:
        df_proc['SUMA_EDU_PADRES'] = df_proc['F_EDUCACIONPADRE'] + df_proc['F_EDUCACIONMADRE']
        df_proc['MAX_EDU_PADRES'] = df_proc[['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']].max(axis=1)

    # C. Ingeniería de Texto (Programas Académicos)
    # Si la columna del programa existe, contamos cuántas veces aparece (Frecuencia)
    # Esto ayuda a distinguir programas populares (Derecho, Medicina) de otros más raros
    categorical_cols = df_proc.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if col != 'RENDIMIENTO_GLOBAL' and col != 'ID':
            # Frequency Encoding: Reemplazamos el texto por qué tan común es
            freq = df_proc[col].value_counts(normalize=True)
            df_proc[f'{col}_FREQ'] = df_proc[col].map(freq)

            # Mantenemos también el LabelEncoder original
            lbl = LabelEncoder()
            df_proc[col] = lbl.fit_transform(df_proc[col].astype(str))

    # Eliminar ID
    if 'ID' in df_proc.columns:
        df_proc = df_proc.drop('ID', axis=1)

    return df_proc

print("Función de Preprocesamiento MEJORADA lista.")

Función de Preprocesamiento MEJORADA lista.


In [18]:
# Procesar Train y Test
train_proc = preprocess_data(train, is_train=True)
test_proc = preprocess_data(test, is_train=False)

# Separar variables predictoras (X) y objetivo (y)
X = train_proc.drop('RENDIMIENTO_GLOBAL', axis=1)
y = train_proc['RENDIMIENTO_GLOBAL'].map(target_map) # Convertimos texto a 0,1,2,3

# Asegurar que el test tenga las mismas columnas (ordenadas igual)
X_test = test_proc[X.columns]

print("Datos procesados y listos para entrenar.")
print(f"X shape: {X.shape}, X_test shape: {X_test.shape}")

Datos procesados y listos para entrenar.
X shape: (692500, 29), X_test shape: (296786, 29)


## Accuracy

In [19]:
# --- BLOQUE 6 AJUSTADO (TUNING) ---

# Configuración del Modelo XGBoost
# Cambios: learning_rate más bajo (0.02) y más profundidad (10)
clf = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=4,
    n_estimators=3000,   # Aumentamos para compensar el learning rate bajo
    learning_rate=0.02,  # Aprende más lento para encontrar mínimos locales mejores
    max_depth=10,        # Un poco más profundo para capturar interacciones complejas
    min_child_weight=3,  # Evita que el modelo aprenda de grupos de estudiantes muy pequeños (ruido)
    subsample=0.75,      # Variación aleatoria de filas
    colsample_bytree=0.7, # Variación aleatoria de columnas
    random_state=42,
    n_jobs=-1,
    tree_method='hist',
    early_stopping_rounds=70 # Un poco más de paciencia
)

# Validación Cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds_test_folds = []
scores = []

print("Iniciando entrenamiento PRO (más lento, mejor precisión)...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    clf.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        verbose=False
    )

    val_preds = clf.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, val_preds)
    scores.append(acc)
    print(f"--> Fold {fold+1} Accuracy: {acc:.5f}")

    preds_test_folds.append(clf.predict(X_test))

print(f"\nAccuracy Promedio Global: {np.mean(scores):.5f}")

Iniciando entrenamiento PRO (más lento, mejor precisión)...
--> Fold 1 Accuracy: 0.43939
--> Fold 2 Accuracy: 0.44019
--> Fold 3 Accuracy: 0.43753
--> Fold 4 Accuracy: 0.44004
--> Fold 5 Accuracy: 0.43903

Accuracy Promedio Global: 0.43924


In [21]:
# Votación Mayoritaria (Ensemble)
# Si 3 modelos dicen "alto" y 2 "medio", gana "alto".
print("Generando predicciones finales...")

# Convertir lista de arrays a matriz
preds_matrix = np.column_stack(preds_test_folds)

# Calcular la moda fila por fila
final_preds_idx, _ = mode(preds_matrix, axis=1)
final_preds_idx = final_preds_idx.ravel()

# Convertir números de vuelta a texto (0 -> 'bajo', etc.)
final_preds_str = [inv_target_map[x] for x in final_preds_idx]

# Crear DataFrame de submission
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': final_preds_str
})

# Guardar CSV
submission.to_csv('submission_xgboost_mejorado.csv', index=False)
print("¡Archivo 'submission_xgboost_mejorado.csv' generado con éxito!")
print(submission.head())

Generando predicciones finales...
¡Archivo 'submission_xgboost_mejorado.csv' generado con éxito!
       ID RENDIMIENTO_GLOBAL
0  550236               alto
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo


# Competencia, accurancy

In [22]:
!kaggle competitions submit \
  -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
  -f submission_xgboost_mejorado.csv \
  -m "Submission XGBoost mejorado"


100% 4.06M/4.06M [00:00<00:00, 6.28MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia